# Module biogeme.expressions 

## Examples of use of each function

This webpage is for programmers who need examples of use of the functions of the module. The examples are designed to illustrate the syntax. They do not correspond to any meaningful model. For examples of models, visit  [biogeme.epfl.ch](http://biogeme.epfl.ch).

In [1]:
import datetime
print(datetime.datetime.now())

2022-01-07 15:43:19.607715


In [2]:
import biogeme.version as ver
print(ver.getText())

biogeme 3.2.9a [2022-01-07]
Version entirely written in Python
Home page: http://biogeme.epfl.ch
Submit questions to https://groups.google.com/d/forum/biogeme
Michel Bierlaire, Transport and Mobility Laboratory, Ecole Polytechnique Fédérale de Lausanne (EPFL)



In [3]:
import numpy as np
import pandas as pd

In [4]:
import biogeme.expressions as ex
import biogeme.database as db
import biogeme.exceptions as excep
import biogeme.models as models
from biogeme import tools
import biogeme.messaging as msg

We first create a small database

In [5]:
df = pd.DataFrame({'Person': [1, 1, 1, 2, 2],
                   'Exclude': [0, 0, 1, 0, 1],
                   'Variable1': [10, 20, 30, 40, 50],
                   'Variable2': [100, 200, 300, 400, 500],
                   'Choice': [2, 2, 3, 1, 2],
                   'Av1': [0, 1, 1, 1, 1],
                   'Av2': [1, 1, 1, 1, 1],
                   'Av3': [0, 1, 1, 1, 1]})
df

,Person,Exclude,Variable1,Variable2,Choice,Av1,Av2,Av3
0,1,0,10,100,2,0,1,0
1,1,0,20,200,2,1,1,1
2,1,1,30,300,3,1,1,1
3,2,0,40,400,1,1,1,1
4,2,1,50,500,2,1,1,1


In [6]:
myData = db.Database('test', df)

The following type of expression is a literal called Variable that corresponds to an entry in the database.

In [7]:
Person = ex.Variable('Person')
Variable1 = ex.Variable('Variable1')
Variable2 = ex.Variable('Variable2')
Choice = ex.Variable('Choice')
Av1 = ex.Variable('Av1')
Av2 = ex.Variable('Av2')
Av3 = ex.Variable('Av3')

It is possible to add a new column to the database, that creates a new variable that can be used in expressions.

In [8]:
newvar_b = ex.DefineVariable('newvar_b',
                           Variable1 + Variable2,
                           myData)

It is equivalent to the following Pandas statement

In [9]:
myData.data['newvar_p'] = myData.data['Variable1'] + myData.data['Variable2']
myData.data

,Person,Exclude,Variable1,Variable2,Choice,Av1,Av2,Av3,newvar_b,newvar_p
0,1,0,10,100,2,0,1,0,110.0,110
1,1,0,20,200,2,1,1,1,220.0,220
2,1,1,30,300,3,1,1,1,330.0,330
3,2,0,40,400,1,1,1,1,440.0,440
4,2,1,50,500,2,1,1,1,550.0,550


**Do not use chaining comparison expressions with Biogeme. Not only it does not provide the expected expression, but it does not trigger a warning or an exception.**

In [10]:
my_expression = (200 <= Variable2 <= 400)
print(my_expression)

(Variable2 <= `400`)


The reason is that Python executes `200 <= Variable2 <= 400` as `(200 <= Variable2) and (Variable2 <= 400)`. The `and` operator cannot be overloaded in Python. Therefore, it does not return a Biogeme expression. Note that Pandas does not allow chaining either, and has implemented a `between` function instead. 

In [11]:
myData.data['chaining_p'] = myData.data['Variable2'].between(200, 400)

In [12]:
myData.data

,Person,Exclude,Variable1,Variable2,Choice,Av1,Av2,Av3,newvar_b,newvar_p,chaining_p
0,1,0,10,100,2,0,1,0,110.0,110,False
1,1,0,20,200,2,1,1,1,220.0,220,True
2,1,1,30,300,3,1,1,1,330.0,330,True
3,2,0,40,400,1,1,1,1,440.0,440,True
4,2,1,50,500,2,1,1,1,550.0,550,False


The following type of expression is another literal, corresponding to an unknown parameter. 

In [13]:
beta1 = ex.Beta('beta1', 0.2, None, None, 0)
beta2 = ex.Beta('beta2', 0.4, None, None, 0)
beta3 = ex.Beta('beta3', 1, None, None, 1)
beta4 = ex.Beta('beta4', 0, None, None, 1)

Arithmetic operators are overloaded to allow standard manipulations of expressions. The first expression is $$e_1 = 2  \beta_1 - \frac{\exp(-\beta_2)}{\beta_2 (\beta_3 \geq \beta_4) + \beta_1 (\beta_3 < \beta_4)},$$
where $(\beta_2 \geq \beta_1)$ equals 1 if $\beta_2 \geq \beta_1$ and 0 otherwise.

In [14]:
expr1 = 2 * beta1 - ex.exp(-beta2) / (beta2 * (beta3 >= beta4) + beta1 * (beta3 < beta4))
print(expr1)

((`2` * beta1(0.2)) - (exp((-beta2(0.4))) / ((beta2(0.4) * (beta3(1) >= beta4(0))) + (beta1(0.2) * (beta3(1) < beta4(0))))))


The evaluation of expressions can be done in two ways. For simple expressions, the fonction getValue(), implemented in Python, returns the value of the expression.  

In [15]:
expr1.getValue()

-1.275800115089098

It is possible to modify the values of the parameters

In [16]:
newvalues = {'beta1': 1, 'beta2': 2, 'beta3': 3, 'beta4': 2}
expr1.changeInitValues(newvalues)
expr1.getValue()

1.9323323583816936

The function getValue_c() is implemented in C++, and works for any expression.

In [17]:
expr1.getValue_c()

1.9323323583816936

It actually calls the function getValueAndDerivates(), and returns its first output (without calculating the derivatives).

In [18]:
f, g, h, bhhh = expr1.getValueAndDerivatives()

In [19]:
f

1.9323323583816936

In [20]:
g

array([2.        , 0.10150146])

In [21]:
h

array([[ 0.       ,  0.       ],
       [ 0.       , -0.1691691]])

In [22]:
bhhh

array([[4.        , 0.20300292],
       [0.20300292, 0.01030255]])

Note that the BHHH matrix is the outer product of the gradient with itself.

In [23]:
np.outer(g, g)

array([[4.        , 0.20300292],
       [0.20300292, 0.01030255]])

If the derivatives are not needed, their calculation can be skipped. Here, we calculate the gradient, but not the hessian.

In [24]:
expr1.getValueAndDerivatives(gradient=True, hessian=False, bhhh=False)

(1.9323323583816936, array([2.        , 0.10150146]), None, None)

It can also generate a function that takes the value of the parameters as argument, and provides a tuple with the value of the expression and its derivatives. By default, it returns the value of the function, its gradient and its hessian.

In [25]:
the_function = expr1.createFunction()

We evaluate it at one point...

In [26]:
the_function([1, 2])

(1.9323323583816936,
 array([2.        , 0.10150146]),
 array([[ 0.       ,  0.       ],
        [ 0.       , -0.1691691]]))

... and at another point.

In [27]:
the_function([10, -2])

(23.694528049465326,
 array([ 2.        , -1.84726402]),
 array([[0.        , 0.        ],
        [0.        , 1.84726402]]))

We can use it to check the derivatives

In [28]:
logger = msg.bioMessage()
logger.setDebug()
tools.checkDerivatives(the_function, [1, 2], logg=True)

[15:43:20] < Detailed >  x		Gradient	FinDiff		Difference
[15:43:20] < Detailed >  x[0]           	+2.000000E+00	+2.000000E+00	-1.167734E-09
[15:43:20] < Detailed >  x[1]           	+1.015015E-01	+1.015014E-01	+1.629049E-08
[15:43:20] < Detailed >  Row		Col		Hessian	FinDiff		Difference
[15:43:20] < Detailed >  x[0]           	x[0]           	+0.000000E+00	+0.000000E+00	+0.000000E+00
[15:43:20] < Detailed >  x[0]           	x[1]           	+0.000000E+00	+0.000000E+00	+0.000000E+00
[15:43:20] < Detailed >  x[1]           	x[0]           	+0.000000E+00	+0.000000E+00	+0.000000E+00
[15:43:20] < Detailed >  x[1]           	x[1]           	-1.691691E-01	-1.691691E-01	-3.203118E-08


(1.9323323583816936,
 array([2.        , 0.10150146]),
 array([[ 0.       ,  0.       ],
        [ 0.       , -0.1691691]]),
 array([-1.16773435e-09,  1.62904950e-08]),
 array([[ 0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00, -3.20311803e-08]]))

But it is possible to also obtain the BHHH matrix.

In [29]:
the_function = expr1.createFunction(bhhh=True)
the_function([1, 2])

(1.9323323583816936,
 array([2.        , 0.10150146]),
 array([[ 0.       ,  0.       ],
        [ 0.       , -0.1691691]]),
 array([[4.        , 0.20300292],
        [0.20300292, 0.01030255]]))

It can take a database as input, and evaluate the expression  and its derivatives for each entry in the database.
In the following example, as no variable of the database is involved in the expression, the output of the expression is the same for each entry.

In [30]:
expr1.getValueAndDerivatives(database=myData, aggregation=False)

(array([1.93233236, 1.93233236, 1.93233236, 1.93233236, 1.93233236]),
 array([[2.        , 0.10150146],
        [2.        , 0.10150146],
        [2.        , 0.10150146],
        [2.        , 0.10150146],
        [2.        , 0.10150146]]),
 array([[[ 0.       ,  0.       ],
         [ 0.       , -0.1691691]],
 
        [[ 0.       ,  0.       ],
         [ 0.       , -0.1691691]],
 
        [[ 0.       ,  0.       ],
         [ 0.       , -0.1691691]],
 
        [[ 0.       ,  0.       ],
         [ 0.       , -0.1691691]],
 
        [[ 0.       ,  0.       ],
         [ 0.       , -0.1691691]]]),
 array([[[4.        , 0.20300292],
         [0.20300292, 0.01030255]],
 
        [[4.        , 0.20300292],
         [0.20300292, 0.01030255]],
 
        [[4.        , 0.20300292],
         [0.20300292, 0.01030255]],
 
        [[4.        , 0.20300292],
         [0.20300292, 0.01030255]],
 
        [[4.        , 0.20300292],
         [0.20300292, 0.01030255]]]))

If `aggregation`is set to `False`, the results are accumulated as a sum.

In [31]:
expr1.getValueAndDerivatives(database=myData, aggregation=True)

(9.661661791908468,
 array([10.        ,  0.50750731]),
 array([[ 0.        ,  0.        ],
        [ 0.        , -0.84584552]]),
 array([[20.        ,  1.01501462],
        [ 1.01501462,  0.05151273]]))

The following function scans the expression and extracts a dict with all free parameters.

In [32]:
expr1.setOfBetas()

{'beta1', 'beta2'}

Options can be set to extract free parameters, fixed parameters, or both. 

In [33]:
expr1.setOfBetas(free=False, fixed=True)

{'beta3', 'beta4'}

In [34]:
expr1.setOfBetas(free=True, fixed=True)

{'beta1', 'beta2', 'beta3', 'beta4'}

It is possible also to extract an elementary expression from its name.

In [35]:
expr1.getElementaryExpression('beta2')

beta2(2)

Let's consider an expression involving two variables $V_1$ and $V_2$: $$e_2 = 2  \beta_1 V_1 - \frac{\exp(-\beta_2 V_2)}{\beta_2 (\beta_3 \geq \beta_4) + \beta_1 (\beta_3 < \beta_4)},$$
where $(\beta_2 \geq \beta_1)$ equals 1 if $\beta_2 \geq \beta_1$ and 0 otherwise. Note that, in our example, the second term is numerically negligible with respect to the first one.

In [36]:
expr2 = 2 * beta1 * Variable1 - ex.exp(-beta2 * Variable2) / (beta2 * (beta3 >= beta4) + beta1 * (beta3 < beta4))
print(expr2)

(((`2` * beta1(1)) * Variable1) - (exp(((-beta2(2)) * Variable2)) / ((beta2(2) * (beta3(3) >= beta4(2))) + (beta1(1) * (beta3(3) < beta4(2))))))


It is not a simple expression anymore, and only the function `getValue_c` can be invoked. If we try the `getValue`function, it raises an exception.

In [37]:
try:
    expr2.getValue()
except excep.biogemeError as e:
    print(f'Exception raised: {e}')

Exception raised: Evaluating Variable Variable1 requires a database. Use the function getValue_c instead.


In [38]:
expr2.getValue_c(database=myData, aggregation=False)

array([ 20.,  40.,  60.,  80., 100.])

Note that if no database is provided, an exception is raised when the formula contains variables. Indeed, the values of these variables cannot be found anywhere.

In [39]:
try:
    expr2.getValue_c()
except excep.biogemeError as e:
    print(f'Exception raised: {e}')

Exception raised: The database must be provided to audit the variable.


The following function extracts the names of the parameters apprearing in the expression

In [40]:
expr2.setOfBetas(free=True,fixed=True)

{'beta1', 'beta2', 'beta3', 'beta4'}

The list of parameters can also be obtained in the form of a dictionary.

In [41]:
expr2.dictOfBetas(free=True,fixed=True)

{'beta1': beta1(1), 'beta2': beta2(2), 'beta3': beta3(3), 'beta4': beta4(2)}

The list of variables can also be obtained in the form of a dictionary

In [42]:
expr2.dictOfVariables()

{'Variable1': Variable1, 'Variable2': Variable2}

or a set...

In [43]:
expr2.setOfVariables()

{'Variable1', 'Variable2'}

Expressions are defined recursively, using a tree representation. The following function describes the type of the upper most node of the tree.

In [44]:
expr2.getClassName()

'Minus'

The signature is a formal representation of the expression, assigning identifiers to each node of the tree, and representing them starting from the leaves. It is easy to parse, and is passed to the C++ implementation. 

In [45]:
expr2.getSignature()

[b'<Numeric>{140176065526272},2',
 b'<Beta>{140175521134912}"beta1"[0],0,0',
 b'<Times>{140176065526176}(2),140176065526272,140175521134912',
 b'<Variable>{140175521130160}"Variable1",6,2',
 b'<Times>{140176065527232}(2),140176065526176,140175521130160',
 b'<Beta>{140175521134144}"beta2"[0],1,1',
 b'<UnaryMinus>{140176065527184}(1),140175521134144',
 b'<Variable>{140175521131840}"Variable2",7,3',
 b'<Times>{140176065527136}(2),140176065527184,140175521131840',
 b'<exp>{140176065527088}(1),140176065527136',
 b'<Beta>{140175521134144}"beta2"[0],1,1',
 b'<Beta>{140175521135776}"beta3"[1],2,0',
 b'<Beta>{140175521134672}"beta4"[1],3,1',
 b'<GreaterOrEqual>{140176065525888}(2),140175521135776,140175521134672',
 b'<Times>{140176065525360}(2),140175521134144,140176065525888',
 b'<Beta>{140175521134912}"beta1"[0],0,0',
 b'<Beta>{140175521135776}"beta3"[1],2,0',
 b'<Beta>{140175521134672}"beta4"[1],3,1',
 b'<Less>{140176065758688}(2),140175521135776,140175521134672',
 b'<Times>{140176065758736}

The elementary expressions are
- free parameters,
- fixed parameters,
- random variables (for numerical integration),
- draws (for Monte-Carlo integration), and
- variables from the database.

The following function extracts all elementary expressions from a list of formulas, give them a unique numbering, and return them organized by group, as defined above (with the exception of the variables, that are directly available in the database).

In [46]:
collectionOfFormulas = [expr1, expr2]
(elementaryExpressionIndex,
    allFreeBetas,freeBetaNames,
    allFixedBetas,
    fixedBetaNames,
    allRandomVariables,
    randomVariableNames,
    allDraws,
    drawNames) =\
ex.defineNumberingOfElementaryExpressions(collectionOfFormulas,
                                         list(myData.data.columns))

Unique numbering for all elementary expressions

In [47]:
elementaryExpressionIndex

{'beta1': 0,
 'beta2': 1,
 'beta3': 2,
 'beta4': 3,
 'Person': 4,
 'Exclude': 5,
 'Variable1': 6,
 'Variable2': 7,
 'Choice': 8,
 'Av1': 9,
 'Av2': 10,
 'Av3': 11,
 'newvar_b': 12,
 'newvar_p': 13,
 'chaining_p': 14}

In [48]:
allFreeBetas

{'beta1': beta1(1), 'beta2': beta2(2)}

Each elementary expression has two ids. One unique across all elementary expressions, and one unique within each specific group

In [49]:
[(i.uniqueId, i.betaId) for k, i in allFreeBetas.items()]

[(0, 0), (1, 1)]

In [50]:
freeBetaNames

['beta1', 'beta2']

In [51]:
allFixedBetas

{'beta3': beta3(3), 'beta4': beta4(2)}

In [52]:
[(i.uniqueId, i.betaId) for k, i in allFixedBetas.items()]

[(2, 0), (3, 1)]

In [53]:
fixedBetaNames

['beta3', 'beta4']

In [54]:
allRandomVariables

{}

Monte Carlo integration is based on draws. 

In [55]:
myDraws = ex.bioDraws('myDraws', 'UNIFORM')
expr3 = ex.MonteCarlo(myDraws * myDraws)

In [56]:
print(expr3)

MonteCarlo((bioDraws("myDraws", "UNIFORM") * bioDraws("myDraws", "UNIFORM")))


Note that draws are not random variables, used for numerical integration.

In [57]:
expr3.dictOfRandomVariables()

{}

The following function reports the draws involved in an expression.

In [58]:
expr3.dictOfDraws()

{'myDraws': 'UNIFORM'}

The expression is a Monte-Carlo integration.

In [59]:
expr3.getClassName()

'MonteCarlo'

Note that the draws are associated with a database. Therefore, the evaluation of expressions involving Monte Carlo integration can only be done on a database. If none is provided, an exception is raised.

In [60]:
try:
    expr3.getValue_c(numberOfDraws=100000)
except excep.biogemeError as e:
    print(f'Exception raised: {e}')

Exception raised: An expression involving MonteCarlo integration must be associated with a database.


Here is its value. It is an approximation of $\int_0^1 x^2 dx=\frac{1}{3}$.

In [61]:
expr3.getValue_c(database=myData, numberOfDraws=100000)

array([0.33456742, 0.33308162, 0.33448306, 0.33275812, 0.33448251])

Here is its signature.

In [62]:
expr3.getSignature()

[b'<bioDraws>{140176065766464}"myDraws",0,0',
 b'<bioDraws>{140176065766464}"myDraws",0,0',
 b'<Times>{140176065767568}(2),140176065766464,140176065766464',
 b'<MonteCarlo>{140176065767184}(1),140176065767568']

The same integral can be calculated using numerical integration, declaring a random variable. 

In [63]:
omega = ex.RandomVariable('omega')

Numerical integration calculates integrals between $-\infty$ and $+\infty$. Here, the interval being $[0,1]$, a change of variables is required.

In [64]:
a = 0
b = 1
x = a + (b - a) / ( 1 + ex.exp(-omega))
dx = (b - a) * ex.exp(-omega) * (1 + ex.exp(-omega))**(-2) 
integrand = x * x
expr4 = ex.Integrate(integrand * dx /(b - a), 'omega')

In this case, omega is a random variable.

In [65]:
expr4.dictOfRandomVariables()

{'omega': omega}

In [66]:
print(expr4)

Integrate(((((`0` + (`1` / (`1` + exp((-omega))))) * (`0` + (`1` / (`1` + exp((-omega)))))) * ((`1` * exp((-omega))) * ((`1` + exp((-omega))) ** `-2`))) / `1`), "omega")


Calculating its value requires the C++ implementation.

In [67]:
expr4.getValue_c(myData)

0.3333323120662823

We illustrate now the Elem function. It takes two arguments: a dictionary, and a formula for the key. For each entry in the database, the formula is evaluated, and its result identifies which formula in the dictionary should be evaluated.
Here is 'Person' is 1, the expression is $$e_1=2  \beta_1 - \frac{\exp(-\beta_2)}{\beta_3 (\beta_2 \geq \beta_1)},$$ and if 'Person' is 2, the expression is $$e_2=2 \beta_1  V_1 - \frac{\exp(-\beta_2 V_2) }{ \beta_3  (\beta_2 \geq \beta_1)}.$$ As it is a regular expression, it can be included in any formula. Here, we illustrate it by dividing the result by 10.

In [68]:
elemExpr = ex.Elem({1: expr1, 2: expr2}, Person) 
expr5 =  elemExpr / 10
print(expr5)

({{1:((`2` * beta1(1)) - (exp((-beta2(2))) / ((beta2(2) * (beta3(3) >= beta4(2))) + (beta1(1) * (beta3(3) < beta4(2)))))), 2:(((`2` * beta1(1)) * Variable1) - (exp(((-beta2(2)) * Variable2)) / ((beta2(2) * (beta3(3) >= beta4(2))) + (beta1(1) * (beta3(3) < beta4(2))))))}[Person] / `10`)


In [69]:
expr5.dictOfVariables()

{'Variable1': Variable1, 'Variable2': Variable2, 'Person': Person}

In [70]:
expr5.getValue_c(database=myData)

array([ 0.19323324,  0.19323324,  0.19323324,  8.        , 10.        ])

The next expression is simply the sum of multiples expressions. The argument is a list of expressions. 

In [71]:
expr6 = ex.bioMultSum([expr1, expr2, expr4])

In [72]:
print(expr6)

bioMultSum(((`2` * beta1(1)) - (exp((-beta2(2))) / ((beta2(2) * (beta3(3) >= beta4(2))) + (beta1(1) * (beta3(3) < beta4(2)))))), (((`2` * beta1(1)) * Variable1) - (exp(((-beta2(2)) * Variable2)) / ((beta2(2) * (beta3(3) >= beta4(2))) + (beta1(1) * (beta3(3) < beta4(2)))))), Integrate(((((`0` + (`1` / (`1` + exp((-omega))))) * (`0` + (`1` / (`1` + exp((-omega)))))) * ((`1` * exp((-omega))) * ((`1` + exp((-omega))) ** `-2`))) / `1`), "omega"))


In [73]:
expr6.getValue_c(database=myData, numberOfDraws=100000)

array([ 22.26566467,  42.26566467,  62.26566467,  82.26566467,
       102.26566467])

We now illustrate how to calculate a logit model, that is $$ \frac{y_1 e^{V_1}}{y_0 e^{V_0}+y_1 e^{V_1}+y_2 e^{V_2}}$$ where $V_0=-\beta_1$, $V_1=-\beta_2$ and $V_2=-\beta_1$, and $y_i = 1$, $i=1,2,3$.

In [74]:
V = {0: -beta1, 1: -beta2, 2: -beta1}
av = {0: 1, 1: 1, 2: 1}
expr7 = ex.LogLogit(V, av, 1)

In [75]:
expr7.getValue()

-1.861994804058251

If the alternative is not in the choice set, an exception is raised.

In [76]:
expr7_wrong = ex.LogLogit(V, av, 3)
try:
    expr7_wrong.getValue()
except excep.biogemeError as e:
    print(f'Exception: {e}')

Exception: Alternative 3 for not appear in the list of utility functions: dict_keys([0, 1, 2])


It is actually better to use the C++ implementation, available in the module models

In [77]:
expr8 = models.loglogit(V, av, 1)

In [78]:
expr8.getValue_c(database=myData)

array([-1.8619948, -1.8619948, -1.8619948, -1.8619948, -1.8619948])

As the result is a numpy array, it can be used for any calculation. Here, we show how to calculate the logsum

In [79]:
for v in V.values():
    print(v.getValue_c(database=myData))

[-1. -1. -1. -1. -1.]
[-2. -2. -2. -2. -2.]
[-1. -1. -1. -1. -1.]


In [80]:
logsum = np.log(np.sum([np.exp(v.getValue_c(database=myData)) 
                        for v in V.values()], axis=1))
logsum

array([ 0.60943791, -0.39056209,  0.60943791])

It is possible to calculate the derivative of a formula with respect to a literal: $$e_9=\frac{\partial e_8}{\partial \beta_2}.$$

In [81]:
expr9 = ex.Derive(expr8, 'beta2')

In [82]:
expr9.getValue_c(database=myData)

array([-0.8446376, -0.8446376, -0.8446376, -0.8446376, -0.8446376])

Biogeme also provides an approximation of the CDF of the normal distribution: $$e_{10}= \frac{1}{{\sigma \sqrt {2\pi } }}\int_{-\infty}^t e^{{{ - \left( {x - \mu } \right)^2 } \mathord{\left/ {\vphantom {{ - \left( {x - \mu } \right)^2 } {2\sigma ^2 }}} \right. } {2\sigma ^2 }}}dx$$

In [83]:
expr10 = ex.bioNormalCdf(Variable1 / 10 - 1)

In [84]:
expr10.getValue_c(database=myData)

array([0.5       , 0.84134475, 0.97724987, 0.9986501 , 0.99996833])

Min and max operators are also available. To avoid any ambiguity with the Python operator, they are called bioMin and bioMax. 

In [85]:
expr11 = ex.bioMin(expr5, expr10)
expr11.getValue_c(database=myData)

array([0.19323324, 0.19323324, 0.19323324, 0.9986501 , 0.99996833])

In [86]:
expr12 = ex.bioMax(expr5, expr10)
expr12.getValue_c(database=myData)

array([ 0.5       ,  0.84134475,  0.97724987,  8.        , 10.        ])

For the sake of efficiency, it is possible to specify explicitly a linear function, where each term is the product of a parameter and a variable.

In [87]:
terms = [(beta1, ex.Variable('Variable1')),
         (beta2, ex.Variable('Variable2')),
         (beta3, ex.Variable('newvar'))]

In [88]:
expr13 = ex.bioLinearUtility(terms)

In [89]:
expr13.getValue_c(database=myData)

[15:43:21] < Warning >   Variable newvar not found in the database.


biogemeError: Variable newvar not found in the database.

In terms of specification, it is equivalent to the expression below. But the calculation of the derivates is more efficient, as the linear structure of the specification is exploited.

In [ ]:
expr13bis = beta1 * Variable1 + beta2 * Variable2 + beta3 * newvar

In [ ]:
expr13bis.getValue_c(database=myData)

A Pythonic way to write a linear utility function

In [ ]:
variables = ['v1', 'v2', 'v3', 'cost', 'time', 'headway']
coefficients = {f'{v}': ex.Beta(f'beta_{v}', 0, None, None, 0) 
                for v in variables}
terms = [coefficients[v] * ex.Variable(v) for v in variables]
util = sum(terms)
print(util)

If the data is organized a panel data, it means that several rows correspond to the same individual. The expression `PanelLikelihoodTrajectory` calculates the product of the expression evaluated for each row. If Monte Carlo integration is involved, the same draws are used for each them.

Our database contains 5 observations.

In [ ]:
myData.getSampleSize()

In [ ]:
myData.panel('Person')

Once the data has been labeled as "panel", it is considered that there are only two series of observations, corresponsing to each person. Each of these observation is associated with several rows of observations.

In [ ]:
myData.getSampleSize()

If we try to evaluate again the integral $\int_0^1 x^2 dx=\frac{1}{3}$, an exception is raised.

In [ ]:
try:
    expr3.getValue_c(database=myData)
except excep.biogemeError as e:
    print(f'Exception: {e}')

This is detected my the `audit` function, called before the expression is evaluated.

In [ ]:
expr3.audit(database=myData)

We now evaluate an expression for panel data.

In [ ]:
c1 = ex.bioDraws('draws1', 'NORMAL_HALTON2')
c2 = ex.bioDraws('draws2', 'NORMAL_HALTON2')
U1 = ex.Beta('beta1', 0, None, None, 0) * Variable1 + 10 * c1
U2 = ex.Beta('beta2', 0, None, None, 0) * Variable2 + 10 * c2
U3 = 0
U = {1: U1, 2: U2, 3: U3}
av = {1: Av1, 2: Av2, 3: Av3}
expr14 = ex.log(ex.MonteCarlo(ex.PanelLikelihoodTrajectory(models.logit(U, av, Choice))))

In [ ]:
expr14.getValue_c(database=myData, numberOfDraws=100000)

In [ ]:
expr14.getValueAndDerivatives(database=myData, numberOfDraws=100000, gradient=True, hessian=True, aggregation=False)

In [ ]:
expr14.getValueAndDerivatives(database=myData, numberOfDraws=100000, gradient=True, hessian=True, aggregation=True)

A Python function can also be obtained for this expression. Note that it is available only for the aggregated version, summing over the database.

In [ ]:
the_function = expr14.createFunction(database=myData, numberOfDraws=100000, gradient=True, hessian=True)

In [ ]:
the_function([0, 0])

In [ ]:
the_function([0.1, 0.1])

# Signatures

The Python library communicates the expressions to the C++ library using a syntax called a "signature". We describe and illustrate now the signature for each expression. Each expression is identified by an identifier provided by Python using the function 'id'. 

In [ ]:
id(expr1)

## Numerical expression

&lt;Numeric&gt;{identifier},value

In [ ]:
ex.Numeric(0).getSignature()

## Beta parameters

&lt;Beta&gt;{identifier}"name"[status],uniqueId,betaId'
where 
- status is 0 for free parameters, and non zero for fixed parameters,
- uniqueId is a unique index given by Biogeme to all elementary expressions,
- betaId is a unique index given by Biogeme to all free parameters, and to all fixed parameters.

In [ ]:
beta1.getSignature()

In [ ]:
beta3.getSignature()

## Variables

&lt;Variable&gt;{identifier}"name",uniqueId,variableId 
where
- uniqueId is a unique index given by Biogeme to all elementary expressions,
- variableId is a unique index given by Biogeme to all variables.


In [ ]:
Variable1.getSignature()

## Random variables

&lt;RandomVariable&gt;{identifier}"name",uniqueId,randomVariableId
where
- uniqueId is a unique index given by Biogeme to all elementary expressions,
- randomVariableId is a unique index given by Biogeme to all random variables.

In [ ]:
omega.getSignature()

## Draws

&lt;bioDraws&gt;{identifier}"name",uniqueId,drawId
where
- uniqueId is a unique index given by Biogeme to all elementary expressions,
- drawId is a unique index given by Biogeme to all draws.


In [ ]:
myDraws.getSignature()

## General expression

<code>&lt;operator&gt;{identifier}(numberOfChildren),idFirstChild,idSecondChild,idThirdChild,</code> etc...
where the number of identifiers given after the comma matches the reported number of children. 

Specific examples are reported below.

### Binary operator

<code>&lt;operator&gt;{identifier}(2),idFirstChild,idSecondChild </code>
where operator is one of: 
    - 'Plus'
    - 'Minus'
    - 'Times'
    - 'Divide'
    - 'Power'
    - 'bioMin'
    - 'bioMax'
    - 'And'
    - 'Or'
    - 'Equal'
    - 'NotEqual'
    - 'LessOrEqual'
    - 'GreaterOrEqual'
    - 'Less'
    - 'Greater'


In [ ]:
sum = beta1 + Variable1

In [ ]:
sum.getSignature()

### Unary operator

&lt;operator&gt;{identifier}(1),idChild, 
where operator is one of: 
    - 'UnaryMinus'
    - 'MonteCarlo'
    - 'bioNormalCdf'
    - 'PanelLikelihoodTrajectory'
    - 'exp'
    - 'log'

In [ ]:
m = -beta1

In [ ]:
m.getSignature()

## LogLogit

&lt;LogLogit&gt;{identifier}(nbrOfAlternatives),chosenAlt,altNumber,utility,availability,altNumber,utility,availability, etc.

In [ ]:
expr7.getSignature()

## Derive

&lt;Derive&gt;{identifier},id of expression to derive,unique index of elementary expression

In [ ]:
expr9.getSignature()

## Integrate

&lt;Integrate&gt;{identifier},id of expression to derive,index of random variable

In [ ]:
expr4.getSignature()

## Elem

&lt;Elem&gt;{identifier}(numberOfExpressions),keyId,value1,expression1,value2,expression2, etc...

where
- keyId is the identifier of the expression calculating the key,
- the number of pairs valuex,expressionx must correspond to the value of numberOfExpressions

In [ ]:
elemExpr.getSignature()

## bioLinearUtility

&lt;bioLinearUtility&gt;{identifier}(numberOfTerms), beta1_exprId, beta1_uniqueId, beta1_name, variable1_exprId, variable1_uniqueId, variable1_name, etc...

where 6 entries are provided for each term:
    - beta1_exprId is the expression id of the beta parameter
    - beta1_uniqueId is the unique id of the beta parameter
    - beta1_name is the name of the parameter
    - variable1_exprId is the expression id of the variable
    - variable1_uniqueId is the unique id of the variable
    - variable1_name is the name of the variable


In [ ]:
expr13.getSignature()